In [1]:
USE_SERVER = True
if USE_SERVER:
    !pip install elasticsearch
    !pip install elasticsearch_dsl
    !pip install pymed
    !pip install gensim
    !pip install torchsummary

  Using cached elasticsearch-7.8.0-py2.py3-none-any.whl (188 kB)
  Using cached elasticsearch_dsl-7.2.1-py2.py3-none-any.whl (53 kB)
  Using cached pymed-0.8.9-py3-none-any.whl (9.6 kB)
  Using cached gensim-3.8.3-cp36-cp36m-manylinux1_x86_64.whl (24.2 MB)
Processing /home/ubuntu/.cache/pip/wheels/a4/9b/d5/85705a7ab783cd6f7bd718f01d3b1396272f30044e3c36401a/smart_open-2.1.0-py3-none-any.whl
  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)


In [2]:
from __future__ import print_function
import argparse
import numpy as np
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import pandas as pd
import utils
import os
from utils import PROJECT_ROOT, DATA_PATH
import yuval_module.paper_source as PaperSource

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

import py_4.get_mesh_vec as get_mesh_vec
import py_3.sim_matrix_3 as sim_matrix_3
import py_4.get_all_features as get_all_features 

from torch.utils.data import Dataset, DataLoader

from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import time

from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

In [3]:
mesh_embed=get_mesh_vec.MeshEmbeddings(PROJECT_ROOT + "data/mesh_data/MeSHFeatureGeneratedByDeepWalk.csv")

In [4]:
FILE = "enriched_labeled_dataset_large" 
if os.path.exists(PROJECT_ROOT + DATA_PATH + FILE):
    print("READING FROM LOCAL")
    if FILE.split(".")[1] == "json":
        df = pd.read_json(PROJECT_ROOT + DATA_PATH + FILE)
    else:
        df = pd.read_csv(PROJECT_ROOT + DATA_PATH + FILE)
    #ps = PaperSource()
else:
    print("PULLING FROM S3")
    ps = sim_matrix_3.load_dataset(FILE)
    df = ps.get_dataset()

df.drop(columns=["last_author_country"],inplace=True)
df.rename(columns={'ORG_STATE':'last_author_country'},inplace=True)

print("FILE PULLED")

PULLING FROM S3
FILE PULLED


In [5]:
cd ..

/home/ubuntu/AYP


In [6]:
auth_core = pd.read_csv("data/train_set_author_names.csv")["0"]
auth_eps = pd.read_csv("data/val_set_author_names.csv")["0"]
auth_usecase = pd.read_csv("data/test_set_author_names.csv")["0"]

In [7]:
selection_train = list(set(df['last_author_name']) - set(auth_usecase))[:4000]
selection_test = list(set(df['last_author_name']) - set(auth_usecase))[4000:4500]

In [8]:
#settings

BATCH_SIZE= 32
EPOCHS = 30
cuda = torch.cuda.is_available()
seed = 42
log_interval = 10
num_workers = 2

#check for cuda
device = torch.device('cuda' if cuda else 'cpu')

torch.manual_seed(seed)
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

device

device(type='cpu')

In [9]:
all_vae = get_all_features.VAE_Features(df)
df_nonan = df[np.invert(df['mesh'].isna())]

'/home/ubuntu/AYP/code/models/names_epochs_2_vectorSize_64_window_2.model' already exits. Using existing model to re-generate results.
'/home/ubuntu/AYP/code/models/co_authors_epochs_2_vectorSize_64_window_2.model' already exits. Using existing model to re-generate results.


In [10]:
class ToyDS(Dataset):
    def __init__(self,df,selection,vae_features = None):
        super().__init__()
        self.df = df[df['last_author_name'].isin(selection)]
        if vae_features is None:
            print("Creating new VAE FEATURES")
            self.vae_features = get_all_features.VAE_Features(self.df)
        else:
            print("Using pre-defined VAE FEATURES")
            self.vae_features = vae_features
        self.features = self.vae_features.get_all_features(self.df)
        print(list(self.vae_features.mesh_features.mesh_missing))
        self.input_dim = self.vae_features.input_dims
            
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        features = self.features[idx]
        return features
    
    def __getvae__(self):
        return self.vae_features

In [11]:
train_set = ToyDS(df_nonan, selection_train, all_vae)
train_loader=DataLoader(dataset= train_set, batch_size = BATCH_SIZE, shuffle = True, num_workers = num_workers)

Using pre-defined VAE FEATURES


/home/ubuntu/AYP/code/py_4/get_all_features.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['co_authors']=df.authors.apply( lambda x: [i['name'] for i in x] )


Defining new scaler
['MESH NAME NOT FOUND: Illicit Drugs', 'MESH NAME NOT FOUND: Male', 'MESH NAME NOT FOUND: Reinforcement, Psychology', 'MESH NAME NOT FOUND: Phospholipid Hydroperoxide Glutathione Peroxidase', 'MESH NAME NOT FOUND: Infections', 'MESH NAME NOT FOUND: Chlorocebus aethiops', 'MESH NAME NOT FOUND: Recognition, Psychology', 'MESH NAME NOT FOUND: Dependency, Psychological', 'MESH NAME NOT FOUND: mTOR Associated Protein, LST8 Homolog', "MESH NAME NOT FOUND: Practice Patterns, Physicians'", 'MESH NAME NOT FOUND: Conditioning, Psychological', 'MESH NAME NOT FOUND: Outcome Assessment, Health Care', 'MESH NAME NOT FOUND: Diet, Healthy', 'MESH NAME NOT FOUND: RNA, Circular', 'MESH NAME NOT FOUND: Outcome and Process Assessment, Health Care', 'MESH NAME NOT FOUND: Inhibition, Psychological', 'MESH NAME NOT FOUND: Phosphoinositide-3 Kinase Inhibitors', 'MESH NAME NOT FOUND: Female', 'MESH NAME NOT FOUND: Confounding Factors, Epidemiologic']


In [66]:
class VAE_GAUSSIAN(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim = train_set.input_dim):
        super(VAE_GAUSSIAN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim

        self.fc1 = nn.Linear(self.input_dim, self.hidden_dim)
        self.fc21 = nn.Linear(self.hidden_dim, self.latent_dim)
        self.fc22 = nn.Linear(self.hidden_dim, self.latent_dim)
        self.fc3 = nn.Linear(self.latent_dim, self.hidden_dim)
        self.fc4 = nn.Linear(self.hidden_dim, self.input_dim)
        self.fc5 = nn.Linear(self.hidden_dim, self.input_dim)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc21(h), self.fc22(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample

    def decode(self, z):
        h = F.relu(self.fc3(z))
        mu = self.fc4(h)
        logvar = self.fc5(h)
        return mu, logvar

    def forward(self, x):
        mu_latent, logvar_latent = self.encode(x.view(-1, self.input_dim))
        z = self.reparameterize(mu_latent, logvar_latent)
        mu_x, logvar_x = self.decode(z)
        return mu_latent, logvar_latent, mu_x, logvar_x
    
def loss_function_gaussian(x, mu_x, logvar_x, mu_latent, logvar_latent):
    # neg log likelihood of x under normal
    loss_rec = np.log(2.0 * np.pi) + logvar_x + (x - mu_x)**2 / (2*torch.exp(logvar_x))
    KLD = -0.5 * torch.sum(1 + logvar_latent - mu_latent.pow(2) - logvar_latent.exp())
    return torch.mean(loss_rec) + KLD

In [67]:
model_gaussian = VAE_GAUSSIAN(hidden_dim=32,latent_dim=2).to(device)
optimizer_gaussian = optim.Adam(model_gaussian.parameters())

In [68]:
def train_epoch_gaussian(tr_loader, model, criterion, optimizer, num_epochs):
    model.train()
    train_log = []
    
    for epoch in range(num_epochs):
        print("Started training epoch no. {}".format(epoch+1))
        train_loss= 0
        for batch_idx, data in enumerate(train_loader):
            data = data.to(device, dtype=torch.float32)
            optimizer.zero_grad()
            mu_latent, logvar_latent, mu_x, logvar_x = model(data)
            loss = criterion(data, mu_x, logvar_x, mu_latent, logvar_latent)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
        print('====> Epoch: {} Average loss: {:.8f}\n'.format(
            epoch+1, train_loss / len(train_loader.dataset)))
        train_log.append({
            'epoch': epoch,
            'train_loss': train_loss / len(train_loader.dataset)})
    return train_log

In [70]:
def test_model_gaussian(test_loader, model):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            data = data.to(device, dtype=torch.float32)
            mu_latent, logvar_latent, mu_x, logvar_x = model(data)
            test_loss += loss_function_gaussian(data, mu_x, logvar_x, mu_latent, logvar_latent).item()
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [71]:
train_epoch_gaussian(train_loader,model_gaussian,loss_function_gaussian,optimizer_gaussian,num_epochs= EPOCHS)

Started training epoch no. 1
====> Epoch: 1 Average loss: 0.06710437

Started training epoch no. 2
====> Epoch: 2 Average loss: 0.06710505

Started training epoch no. 3
====> Epoch: 3 Average loss: 0.06709810

Started training epoch no. 4
====> Epoch: 4 Average loss: 0.06709752

Started training epoch no. 5
====> Epoch: 5 Average loss: 0.06708708

Started training epoch no. 6
====> Epoch: 6 Average loss: 0.06709964

Started training epoch no. 7
====> Epoch: 7 Average loss: 0.06709679

Started training epoch no. 8
====> Epoch: 8 Average loss: 0.06709542

Started training epoch no. 9
====> Epoch: 9 Average loss: 0.06709653

Started training epoch no. 10
====> Epoch: 10 Average loss: 0.06708962

Started training epoch no. 11
====> Epoch: 11 Average loss: 0.06708955

Started training epoch no. 12
====> Epoch: 12 Average loss: 0.06709564

Started training epoch no. 13
====> Epoch: 13 Average loss: 0.06709066

Started training epoch no. 14
====> Epoch: 14 Average loss: 0.06710166

Started tr

[{'epoch': 0, 'train_loss': 0.06710437422197064},
 {'epoch': 1, 'train_loss': 0.06710505465640122},
 {'epoch': 2, 'train_loss': 0.06709810451433046},
 {'epoch': 3, 'train_loss': 0.0670975201881108},
 {'epoch': 4, 'train_loss': 0.06708707719571705},
 {'epoch': 5, 'train_loss': 0.06709963773432416},
 {'epoch': 6, 'train_loss': 0.06709679189559456},
 {'epoch': 7, 'train_loss': 0.06709542422747979},
 {'epoch': 8, 'train_loss': 0.06709652663693245},
 {'epoch': 9, 'train_loss': 0.06708962272627565},
 {'epoch': 10, 'train_loss': 0.06708954922763108},
 {'epoch': 11, 'train_loss': 0.06709563706986467},
 {'epoch': 12, 'train_loss': 0.06709065724266464},
 {'epoch': 13, 'train_loss': 0.06710166200437374},
 {'epoch': 14, 'train_loss': 0.0670884842082221},
 {'epoch': 15, 'train_loss': 0.0670920619672969},
 {'epoch': 16, 'train_loss': 0.0670890144037228},
 {'epoch': 17, 'train_loss': 0.06709734993760823},
 {'epoch': 18, 'train_loss': 0.06709555133607434},
 {'epoch': 19, 'train_loss': 0.06709085023146

In [73]:
test_set = ToyDS(df_nonan, selection_test, vae_features = train_set.__getvae__())
test_loader=DataLoader(dataset= test_set, batch_size = BATCH_SIZE, shuffle = True, num_workers = num_workers)

Using pre-defined VAE FEATURES


/home/ubuntu/AYP/code/py_4/get_all_features.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['co_authors']=df.authors.apply( lambda x: [i['name'] for i in x] )


Using old scaler
['MESH NAME NOT FOUND: Illicit Drugs', 'MESH NAME NOT FOUND: Male', 'MESH NAME NOT FOUND: Reinforcement, Psychology', 'MESH NAME NOT FOUND: Phospholipid Hydroperoxide Glutathione Peroxidase', 'MESH NAME NOT FOUND: Infections', 'MESH NAME NOT FOUND: Chlorocebus aethiops', 'MESH NAME NOT FOUND: Recognition, Psychology', 'MESH NAME NOT FOUND: Dependency, Psychological', 'MESH NAME NOT FOUND: mTOR Associated Protein, LST8 Homolog', "MESH NAME NOT FOUND: Practice Patterns, Physicians'", 'MESH NAME NOT FOUND: Conditioning, Psychological', 'MESH NAME NOT FOUND: Outcome Assessment, Health Care', 'MESH NAME NOT FOUND: Diet, Healthy', 'MESH NAME NOT FOUND: RNA, Circular', 'MESH NAME NOT FOUND: Outcome and Process Assessment, Health Care', 'MESH NAME NOT FOUND: Mitophagy', 'MESH NAME NOT FOUND: Inhibition, Psychological', 'MESH NAME NOT FOUND: Phosphoinositide-3 Kinase Inhibitors', 'MESH NAME NOT FOUND: Female', 'MESH NAME NOT FOUND: Confounding Factors, Epidemiologic']


In [76]:
test_model_gaussian(test_loader, model_gaussian)

====> Test set loss: 0.0670


In [78]:
data = torch.from_numpy(test_set.features).to(device,dtype=torch.float32)
mu_latent, logvar_latent, mu_x, logvar_x = model_gaussian(data)

In [79]:
assert(test_set.features.shape==mu_x.shape)

In [80]:
np.mean(np.square(mu_x.detach().cpu().numpy() - test_set.features))

0.992902788039997